In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import base64
from PIL import Image  
import scipy
from scipy.io import loadmat, savemat

# use DnCNN model in NeCA, NAFlow github
%cd ./Restoration_Reproduce
from models.network_plain import DnCNN 

/home/gurwn/restoration/kaggle/sRGB-Real-Noise-Synthesis


## Benchmark SIDD and Kaggle Submission file

### Path and file name setting
- Pretrained model paths : ``checkpoint_path``

- Submission file : ``output_file``
    - For Kaggle : ``.csv``
    - For SIDD official : ``.mat``

In [ ]:
checkpoint_path = './saves/dncnn_neca_medium_total/models/checkpoint_G3_step_00465000.pth'

"""Output file for Kaggle"""
output_file = 'SubmitSrgb_neca_mecium_total.csv'

"""Output file for SIDD official"""
# output_file = 'SubmitSrgb_neca_mecium_total.mat'            #  When submitting to the server, submit as "SubmitSrgb.mat"

### Load the Denoising model

In [ ]:
def load_checkpoint(checkpoint_path, model):
    """load checkpoint"""
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()  
    return model

def my_srgb_denoiser(x, checkpoint_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load the DnCNN model
    model = DnCNN(in_nc=3, out_nc=3, nc=64, act_mode='BR').to(device)       # Modify based on your model's architecture
    model = load_checkpoint(checkpoint_path, model)

    # Convert Image -> tensor 
    x_tensor = torch.from_numpy(x).permute(2, 0, 1).unsqueeze(0).float().div(255.0).to(device)  # Normalize to [0, 1]

    # Perform denoising
    with torch.no_grad():
        denoised_tensor = model(x_tensor)
    
    # Convert tensor -> numpy
    denoised_array = (
        denoised_tensor.squeeze().cpu().clamp(0, 1).mul(255.0).byte().permute(1, 2, 0).numpy()
    )

    return denoised_array

### Load the Benchmark dataset

In [ ]:
# Input Benchmark file path
input_file = './Benchmark/BenchmarkNoisyBlocksSrgb.mat'

# Read inputs
key = 'BenchmarkNoisyBlocksSrgb'
inputs = loadmat(input_file)
inputs = inputs[key]
print(f'inputs.shape = {inputs.shape}')     # inputs.shape = (40, 32, 256, 256, 3)
print(f'inputs.dtype = {inputs.dtype}')     # inputs.dtype = uint8

**Optional)** Convert benchmark ``.mat`` file to ``.png`` file
- Not necessary X

In [ ]:
# def save_image_blocks(inputs, output_dir):
#     """Saving blocks of images in the format (40, 32, 256, 256, 3) as individual image files"""
#     os.makedirs(output_dir, exist_ok=True)

#     # Image block (i : 0~39, j : 0~31)
#     for i in range(inputs.shape[0]):
#         for j in range(inputs.shape[1]):
#             # (256, 256, 3) size of block
#             img_block = inputs[i, j, :, :, :]
            
#             # Conver numpy -> img
#             img = Image.fromarray(img_block)
            
#             output_path = os.path.join(output_dir, f'low_block_{i}_{j}.png')
            
#             img.save(output_path)
#             print(f"Image save path : {output_path}")

# output_dir = '/home/gurwn/restoration/kaggle/dataset/image'
# save_image_blocks(inputs, output_dir)

### Denoising 

In [ ]:
"""For Kaggle submission(csv)"""
def array_to_base64string(x):
    array_bytes = x.tobytes()
    base64_bytes = base64.b64encode(array_bytes)
    base64_string = base64_bytes.decode('utf-8')
    return base64_string

def base64string_to_array(base64string, array_dtype, array_shape):
    decoded_bytes = base64.b64decode(base64string)
    decoded_array = np.frombuffer(decoded_bytes, dtype=array_dtype)
    decoded_array = decoded_array.reshape(array_shape)
    return decoded_array

In [ ]:
# Save result
"""for Kaggle"""
output_blocks_base64string = []   
"""for SIDD official"""  
# denoise_res = np.zeros_like(inputs)

# Perform denoising for each block
for i in range(inputs.shape[0]):
    for j in range(inputs.shape[1]):
        in_block = inputs[i, j, :, :, :]
        out_block = my_srgb_denoiser(in_block, checkpoint_path)
        assert in_block.shape == out_block.shape
        assert in_block.dtype == out_block.dtype
        """For Kaggle(csv)"""
        out_block_base64string = array_to_base64string(out_block)
        output_blocks_base64string.append(out_block_base64string)
        """for SIDD official  """
        # denoise_res[i, j] = out_block

### Mat submission(for SIDD official server)

In [ ]:
# # Save the result as a MAT file
# savemat(output_file, {'denoise_res': denoise_res})
# print(f'save file : {output_file}')

### CSV submission(for Kaggle server)

In [ ]:
# Save outputs to .csv file
print(f'Saving outputs to {output_file}')
output_df = pd.DataFrame()
n_blocks = len(output_blocks_base64string)
print(f'Number of blocks = {n_blocks}')
output_df['ID'] = np.arange(n_blocks)
output_df['BLOCK'] = output_blocks_base64string

output_df.to_csv(output_file, index=False)

# TODO: Submit the output file SubmitSrgb.csv at 
# kaggle.com/competitions/sidd-benchmark-srgb-psnr
print('TODO: Submit the output file SubmitSrgb.csv at')
print('kaggle.com/competitions/sidd-benchmark-srgb-psnr')

print('Done.')

Saving outputs to SubmitSrgb_DnCNN_neca_medium_total.csv
Number of blocks = 1280
TODO: Submit the output file SubmitSrgb.csv at
kaggle.com/competitions/sidd-benchmark-srgb-psnr
Done.
